# VNINDEX ANALYSIS
## DUY TINH NGUYEN

In [150]:
from googleapiclient.discovery import build
from google.oauth2.credentials import Credentials
from google.oauth2 import service_account
from pandas_datareader import data
import datetime
from bokeh.plotting import figure, show, output_file 

import numpy as np
import matplotlib.pyplot as plt
from mpl_finance import candlestick_ohlc
import mplfinance
import pandas as pd
import matplotlib.dates as mpl_dates


# GET DATA

In [151]:
def get_data (code):
    
    # The ID and range of a sample spreadsheet.
    SAMPLE_SPREADSHEET_ID = '1B-ovqR440h-pKkL7PBVAuiZ8kaqgNjg2ke-ReMeCuXk'
    SAMPLE_RANGE_NAME = 'historyprice!A1:M101'

    SCOPES = ['https://www.googleapis.com/auth/spreadsheets']
    SERVICE_ACCOUNT_FILE = 'keys.json'
    creds = None
    creds = service_account.Credentials.from_service_account_file(
            SERVICE_ACCOUNT_FILE, scopes=SCOPES)


    service = build('sheets', 'v4', credentials=creds)
    MESS = '=IMPORTHTML("https://www.cophieu68.vn/historyprice.php?id=' + code + '";"table";2)'
    val = [[MESS]]
    sheet = service.spreadsheets()
    request = sheet.values().update(spreadsheetId=SAMPLE_SPREADSHEET_ID, range="historyprice!A1", valueInputOption = "USER_ENTERED", body = {"values":val}).execute()

    # Call the Sheets API

    result = sheet.values().get(spreadsheetId=SAMPLE_SPREADSHEET_ID, range=SAMPLE_RANGE_NAME).execute()
    values = result.get('values', [])

    return values

# DATA FILTER

In [152]:
def filter (data):
    output = []
    for i in range (1,len(data)):
        if data[i][0][0] == '#': 
            output.append([datetime.datetime(int(data[i][1][6:10]), int(data[i][1][3:5]), int(data[i][1][0:2])), float(data[i][7]), float(data[i][8]), float(data[i][9]), float(data[i][5][1:-1]), int(data[i][6].replace(',','')), float(data[i][2])])
        # else:
        #     output.append([data[i][0][1:11],np.nan,np.nan,np.nan,np.nan,np.nan,np.nan])
    return output

In [153]:
def plot_candle(code, data, values):
    plt.style.use('ggplot')
    ohlc = data.loc[:, ['Date', 'Open', 'High', 'Low', 'Close']]
    # ohlc['Date'] = pd.to_datetime(ohlc['Date'])
    #tmp = data.loc[:,['Date']]
    #ohlc['Date'] = ohlc['Date'].apply(dates.date2num)
    ohlc['Date'] = np.arange(len(data))
    #ohlc = ohlc.astype(float)

    # Creating Subplots
    fig, ax = plt.subplots()
    
    candlestick_ohlc(ax, ohlc.values, width=0.6, colorup='green', colordown='red', alpha=0.6)

    # Setting labels & titles
    ax.set_xlabel('Date')
    ax.set_ylabel('Price')
    fig.suptitle('Candlestick Chart of '+ code.upper())

    # Formatting Date
    date_format = mpl_dates.DateFormatter('%d-%m-%Y')
    ax.xaxis.set_major_formatter(date_format)
    fig.autofmt_xdate()

    fig.tight_layout()
    fig.set_size_inches(18, 7)
    tmp = []
    x = np.arange(0,len(data),10)
    # print(np.arange(0,len(data),10))
    # print(len(values))
    for i in range(len(x)):
        #print(len(data)-x[i])
        if values[len(data)-x[i]-1][0][0] == '#':
            tmp.append(values[len(data)-x[i]][1])
    
    plt.xticks(x, np.array(tmp))
    plt.plot(x, data.loc[:,['Reference']])
    
    plt.show()


In [154]:
def remove_code_codenote(code):
    data = pd.read_csv('stock_database/codenote.csv')
    data = data.drop(columns = code)
    data.to_csv('stock_database/codenote.csv')
    return data

In [155]:
class candle:
    def __init__(self):
        self.CODE = ''
        #self.VALUES = get_data(self.CODE)
    def check_code_in_csv (self, code):
        data = pd.read_csv('stock_database/codenote.csv')
        if (code in data):
            return True
        else:
            return False
    def r_codenote(self):
        return pd.read_csv('stock_database/codenote.csv')
    def w_codenote(self, code, mess):
        data = pd.read_csv('stock_database/codenote.csv')
        if (not (code in data)):
            data.insert(len(data), code, mess + '\n' + datetime.date.today().strftime("%d/%m/%Y"))
        else:
            for i in range (len(data[code].dropna())):
                data.at[i,code] = data.at[i-1,code]
            data.at[0,code] = mess + '\n' + datetime.date.today().strftime("%d/%m/%Y")
        data.to_csv('stock_database/codenote.csv')
    def cal(self, code):
        if code != self.CODE:
            if(self.check_code_in_csv(code)):
                dat = pd.read_csv('stock_database/' + code + '.csv')
                plot_candle(code, dat, self.VALUES)
            else:
                self.VALUES = get_data(code)
                self.CODE = code
                data = pd.DataFrame(filter(self.VALUES), columns=['Date', 'Open', 'High', 'Low', 'Close','Volume','Reference'])
                data.to_csv('stock_database/' + code + '.csv')
                self.w_codenote(code,'')
                plot_candle(code, data, self.VALUES)
        else:
            data = pd.DataFrame(filter(self.VALUES), columns=['Date', 'Open', 'High', 'Low', 'Close','Volume','Reference'])
            plot_candle(code, data, self.VALUES)
    
    
        

In [156]:
can = candle()

In [149]:
CODE = 'vhc'
can.cal(CODE)

NameError: name 'plot' is not defined

In [83]:
for i in range(len(df['grade'].dropna()), 0,-1):
    df.at[i,'grade'] = df.at[i-1,'grade']
df.at[0,'grade']

In [124]:
data = pd.read_csv('stock_database/codenote.csv')

In [88]:
data = {'Name': ['Jai', 'Princi', 'Gaurav', 'Anuj'],
        'Height': [5.1, 6.2, 5.1, 5.2],
        'Qualification': ['Msc', 'MA', 'Msc', 'Msc']}
  
# Convert the dictionary into DataFrame
df = pd.DataFrame(data)
  
# Using DataFrame.insert() to add a column
df.insert(len(data), "Age", [21, 23, 24, 21], True)
  
# Observe the result
df

,Name,Height,Qualification,Age
0,Jai,5.1,Msc,21
1,Princi,6.2,MA,23
2,Gaurav,5.1,Msc,24
3,Anuj,5.2,Msc,21
